In [2]:
from sklearn.datasets import make_regression

# Generate regression toy data
n_samples = 1000
n_features = 5

X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=42)

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * n_samples)

X_train = X[:train_size]
y_train = y[:train_size]

# Test data
X_test = X[train_size:]
y_test = y[train_size:]

In [5]:
from sklearn.model_selection import cross_val_score


def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [9]:
all_models = {}

# Random Forest

**Feature selection**

**Hyperparameter optimalisatie**

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization


params_ranges = {
    'splitter': (0, 1),  # Numeric mapping of string values: 'best' -> 0, 'random' -> 1
    'max_depth': (1, 20),
    'min_samples_leaf': (1, 10),
    'min_weight_fraction_leaf': (0.0, 0.5),
    'max_features': (0.1, 1),
    'max_leaf_nodes': (10, 100)
}

# Example usage with Decision Trees
model = DecisionTreeRegressor
dt_bo = BayesianOptimization(f=lambda splitter, max_depth, min_samples_leaf, min_weight_fraction_leaf,
                                    max_features, max_leaf_nodes: bo_params_generic(model, {
                                        'splitter': 'best' if splitter < 0.5 else 'random',
                                        'max_depth': round(max_depth),
                                        'min_samples_leaf': round(min_samples_leaf),
                                        'min_weight_fraction_leaf': min_weight_fraction_leaf,
                                        'max_features': max_features,
                                        'max_leaf_nodes': round(max_leaf_nodes)
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model = model(
    splitter='best' if params['splitter'] < 0.5 else 'random',
    max_depth=int(round(params['max_depth'])),
    min_samples_leaf=round(params['min_samples_leaf']),
    min_weight_fraction_leaf=params['min_weight_fraction_leaf'],
    max_features=params['max_features'],
    max_leaf_nodes=int(round(params['max_leaf_nodes']))
)

# Fit the model
best_model.fit(X_train, y_train)


|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_we... | splitter  |
-------------------------------------------------------------------------------------------------
| 1         | 64.01     | 12.57     | 0.1446    | 77.06     | 2.398     | 0.2613    | 0.9289    |
| 2         | 58.44     | 8.551     | 0.6838    | 33.33     | 3.446     | 0.1171    | 0.6876    |
| 3         | 57.91     | 3.285     | 0.9585    | 34.65     | 2.952     | 0.258     | 0.8719    |
| 4         | 62.84     | 19.14     | 0.8052    | 20.08     | 1.248     | 0.3745    | 0.8301    |
| 5         | 42.8      | 6.383     | 0.9121    | 85.0      | 9.198     | 0.01511   | 0.8201    |
| 6         | 58.14     | 8.745     | 0.2175    | 57.97     | 4.526     | 0.2292    | 0.1572    |
| 7         | 54.93     | 14.29     | 0.1635    | 51.11     | 7.019     | 0.1587    | 0.275     |
| 8         | 56.08     | 13.41     | 0.3325    | 70.84     | 3.912     | 0.2247    | 0.1838    |
| 9         | 58.91 

DecisionTreeRegressor(max_depth=12, max_features=0.20137628543170694,
                      max_leaf_nodes=86, min_samples_leaf=4,
                      min_weight_fraction_leaf=0.4742091109198674,
                      splitter='random')

In [ ]:
# Append model to list
all_models['Random forest'] = best_model